In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # Neural network

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
df = pd.read_csv('../input/creditcard.csv')
df.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
print(plt.style.available)

In [ ]:
plt.style.use('ggplot')

In [ ]:
plt.scatter(df[df.Class == 0].Time, df[df.Class == 0].Amount)

In [ ]:
plt.scatter(df[df.Class == 1].Time, df[df.Class == 1].Amount, c=df[df.Class == 1].Class)

In [ ]:
data = df.values

In [ ]:
means = np.mean(df[df.Class == 0].values[:, 1:30], axis=0)
m = len(means)
sigma = np.cov(df[df.Class == 0].values[:, 1:30], rowvar=False)
print(means.shape, sigma.shape)

In [ ]:
def multivariate_gaussian(X, m, s):
    n = len(m) # no features
    X = np.array([X]).T
    m = np.array([m]).T
    pi = 3.14
    det = np.linalg.det(s)
    sinv = np.linalg.inv(s)
    p = 1.0 / (np.power(2 * pi, n / 2) * np.sqrt(det)) 
    p *= np.exp(-1.0 / 2.0 * np.dot(np.dot((X - m).T, sinv), (X - m)))
    return p[0][0]

In [ ]:
# print(sigma.shape)

multivariate_gaussian(data[0, 1:30], means, sigma)

In [ ]:
normals = df[df.Class == 0].values[:, 1:30]
frauds = df[df.Class == 1].values[:, 1:30]

In [ ]:
sum_normals = 0.0
sum_frauds = 0.0

for x in normals[:400]:
    sum_normals += multivariate_gaussian(x, means, sigma)

for x in frauds[:400]:
    sum_frauds += multivariate_gaussian(x, means, sigma)
    
print(sum_normals / 400.0)
print(sum_frauds / 400.0)

In [ ]:
gaussian_normals = []
gaussian_frauds = []

for x in normals[:400]:
    gaussian_normals.append(multivariate_gaussian(x, means, sigma))

for x in frauds[:400]:
    gaussian_frauds.append(multivariate_gaussian(x, means, sigma))

# print(sorted(gaussian_normals))
# print(sorted(gaussian_frauds))

plt.plot(gaussian_normals)
plt.plot(gaussian_frauds, c='blue')
plt.show()

In [ ]:
true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0

threshold = 1e-128

for x in normals:
    if multivariate_gaussian(x, means, sigma) < threshold:
        false_positives += 1
    else:
        true_negatives += 1
        

for x in frauds:
    if multivariate_gaussian(x, means, sigma) < threshold:
        true_positives += 1
    else:
        false_negatives += 1
    
print(true_positives)
print(false_positives)
print(true_negatives)
print(false_negatives)


precision = true_positives * 1.0 / (true_positives + false_positives)
recall = true_positives * 1.0 / (true_positives + false_negatives)
f1_score = 2.0 * (precision * recall) / (precision + recall)
print(precision)
print(recall)
print(f1_score)